In [1]:
import pandas as pd
import json
import re

In [2]:
# Read the TSV file into a pandas DataFrame
pix_df = pd.read_csv("tsv_pixstory_hw2.tsv", sep="\t")


,Unnamed: 0,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,...,translated_titles,translated_narratives,Geographic Information,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
0,0,105133,STY1654473727,8255,USR1628503147,others,13,2. Looking back : On bench,On Bench Once More continues.... While I Simla...,https://image.pixstory.com/Pixstory-image-1628...,...,2. Looking back : On bench,On Bench Once More continues.... While I Simla...,"British Indian Ocean Territory: (-6.0,72.0)",0.20141,0.00094,0.00505,0.00370,0.01475,0.00094,0.14217
1,1,105132,STY1654473498,8601,USR1630424770,others,13,3. Contemporary Afghanistan : The 'Bonn Agreem...,International Community Afghanistan continues....,https://image.pixstory.com/Pixstory-image-1630...,...,3. Contemporary Afghanistan : The 'Bonn Agreem...,International Community Afghanistan continues....,"Islamic Republic of Afghanistan: (33.0,66.0); ...",0.24846,0.00041,0.00949,0.03337,0.04862,0.00053,0.00124
2,2,105131,STY1654473332,28647,USR1646131566,NaN,12,20. A Question Pride : The 'Chushi Gangdrug' r...,Enter Dragon: Chinese Invasion Tibet continues...,https://image.pixstory.com/Pixstory-image-1646...,...,20. A Question Pride : The 'Chushi Gangdrug' r...,Enter Dragon: Chinese Invasion Tibet continues...,"Tibet Autonomous Region: (31.66667,88.0); Rive...",0.03809,0.00019,0.00082,0.00433,0.01085,0.00032,0.00094
3,3,105130,STY1654472717,40917,USR1654101217,male,23,From Undesirable Undeniable - The Cody Rhodes ...,WRESTLING HAS MORE THAN ONE ROYAL FAMILY!! A g...,https://image.pixstory.com/Pixstory-image-1654...,...,From Undesirable Undeniable - The Cody Rhodes ...,WRESTLING HAS MORE THAN ONE ROYAL FAMILY!! A g...,"Austin: (30.26715,-97.74306)",0.00084,0.00022,0.00140,0.00042,0.00169,0.00013,0.00018
4,4,105129,STY1654472629,30966,USR1649551870,NaN,12,Aboriginal elder voice concern proposed Tasman...,It far soon talking proposed guided walk hut T...,https://image.pixstory.com/Pixstory-image-1654...,...,Aboriginal elder voice concern proposed Tasman...,It far soon talking proposed guided walk hut T...,Null,0.00041,0.00002,0.00012,0.00009,0.00020,0.00004,0.00002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94995,94995,130728,STY1661176231,77742,USR1661080107,NaN,19,Lord,Krishna,https://image.pixstory.com/Pixstory-image-1661...,...,Lord,Krishna,Null,0.00172,0.00004,0.00051,0.00014,0.00118,0.00007,0.00004
94996,94996,130727,STY1661176187,78675,USR1661176067,NaN,18,College,College,https://image.pixstory.com/Pixstory-image-1661...,...,College,College,Null,0.86591,0.00035,0.01123,0.00047,0.74393,0.00236,0.00125
94997,94997,130726,STY1661176171,78674,USR1661176023,NaN,18,NaN,NaN,https://image.pixstory.com/Pixstory-image-1661...,...,in,in,Null,0.01341,0.00022,0.00413,0.00067,0.01262,0.00044,0.00051
94998,94998,130725,STY1661175883,78668,USR1661175804,NaN,22,#Gym,#Gym,https://image.pixstory.com/Pixstory-image-1661...,...,#Gym,#Gym,Null,0.00048,0.00005,0.00035,0.00013,0.00043,0.00008,0.00004


In [4]:
pix_df["geometry"]

0        MULTIPOLYGON (((55.42220015400005 21.267509059...
1        MULTIPOLYGON (((68.19780028300005 23.766682128...
2        MULTIPOLYGON (((55.42220015400005 21.267509059...
3                                                      NaN
4                                                      NaN
                               ...                        
94995                                                  NaN
94996                                                  NaN
94997                                                  NaN
94998                                                  NaN
94999                                                  NaN
Name: geometry, Length: 95000, dtype: object

In [82]:

# Define a function to split the geographic information
def split_geographic_info(info):
    if info == "null":
        return [{"location": None, "coordinates": None}]
    
    location_list = info.split(";")
    location_info = []
    
    for loc_str in location_list:
        parts = loc_str.rsplit(':', 1)

        if len(parts) != 2:
            continue

        location = parts[0].strip()

        numbers = re.findall(r"[-+]?\d*\.\d+|[-+]?\d+", parts[1])
        coordinates = [float(num) for num in numbers]  # Convert to list

        location_info.append({"location": location, "coordinates": coordinates})
    
    return location_info

# Apply the function to split the "Geographic Information" column
pix_df["location_info"] = pix_df["Geographic Information"].apply(split_geographic_info)

# Drop the original "Geographic Information" column
pix_df.drop("Geographic Information", axis=1, inplace=True)

# Explode the DataFrame to create separate rows for each location
pix_df = pix_df.explode("location_info")

# Split the location_info column into separate location and coordinates columns
pix_df[["location", "coordinates"]] = pd.json_normalize(pix_df["location_info"])



In [88]:
type(pix_df["coordinates"][0])

list

In [89]:

# Group by "language", "location", and "coordinates", count the occurrences, and reset the index
pix_df = pix_df[pix_df['Tika_Lang_ID'] != '#NAME?']
pix_df["coordinates"] = pix_df["coordinates"].astype(str)

temp = pix_df.groupby(['Tika_Lang_ID', 'location', 'coordinates'], as_index=False).size()

# Reset the index and rename the count column
temp = temp.reset_index().rename(columns={"size": 'count'})
# Drop rows with null values
temp = temp.dropna()

# Sort the DataFrame in descending order based on the count column
temp = temp.sort_values(by='count', ascending=False)

In [96]:
temp["coordinates"].nunique()

4147

In [91]:
# Convert the DataFrame to a list of dictionaries
data_list = temp.to_dict(orient='records')

# Save the data list as a JSON file
with open("language&location.json", "w") as f:
    json.dump(data_list, f, indent=2)